In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd

# Try reading the CSV file with a different encoding
df = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv', header=None,
                 names=["sentiment", "ids", "date", "flag", "user", "text"], encoding='latin1')

In [2]:
df.shape

(1600000, 6)

In [3]:
df.head()

,sentiment,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
df = df.sample(frac=0.25, random_state=42)

In [5]:
df.shape

(400000, 6)

In [6]:
import re
def cleaning_tweets(tweet):
    # Removing unnecessary characters like emojis from the tweets
    tweet = re.sub(r"[^ A-Za-z0-9.!?']", "", tweet)

    # Removing any extra whitespaces
    tweet = re.sub(r" +", " ", tweet)

    # Remove URLs
    tweet = re.sub(r'http\S+', '', tweet)

    # Remove Twitter handles (@mentions)
    tweet = re.sub(r'@\w+', '', tweet)

    # Remove special characters and punctuation
    tweet = re.sub(r'[^\w\s]', '', tweet)

    # Convert to lowercase
    tweet = tweet.lower()

    # Remove numbers
    tweet = re.sub(r'\d+', '', tweet)

    # Remove stopwords (example list)
    stopwords = ['the', 'and', 'is', 'at', 'on', 'for', 'are', 'you']  # Example list of stopwords
    tweet = ' '.join(word for word in tweet.split() if word not in stopwords)

    return tweet

df['clean_text'] = df['text'].apply(cleaning_tweets)


In [7]:
df.head()

,sentiment,ids,date,flag,user,text,clean_text
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!,chrishasboobs ahhh i hope your ok
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo...",misstoriblack cool i have no tweet apps my razr
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...,tiannachaos i know just family drama its lameh...
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...,school email wont open i have geography stuff ...
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem,upper airways problem


In [8]:
df.shape

(400000, 7)

In [9]:
finalData = df[['sentiment','clean_text']]
X = finalData['clean_text']
y = finalData['sentiment']

In [10]:
X.shape

(400000,)

In [15]:
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense,Conv1D,Flatten,Embedding,MaxPool1D,concatenate,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard,EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.initializers import he_normal, glorot_normal
from keras.regularizers import l2
from sklearn.model_selection import train_test_split

In [16]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)
top_words = 10000  # Define the maximum number of words to use
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)


max_sequence_length =5000 
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)# Pad sequences

X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

In [17]:
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [18]:
model = Sequential()
model.add(Embedding(input_dim=top_words, output_dim=50, input_length=max_sequence_length))

# Convolutional layers
model.add(Conv1D(16, 3, activation="relu", kernel_initializer=he_normal(), kernel_regularizer=l2()))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(32, activation="relu", kernel_initializer=he_normal()))
model.add(Dense(2, activation="softmax", kernel_initializer=glorot_normal()))  # 2 units for binary classification

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train_one_hot, epochs=2, batch_size=64, validation_split=0.2)

model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/2
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 953s 238ms/step - accuracy: 0.7307 - loss: 0.5804 - val_accuracy: 0.7796 - val_loss: 0.4789
Epoch 2/2
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 982s 238ms/step - accuracy: 0.8071 - loss: 0.4391 - val_accuracy: 0.7813 - val_loss: 0.4830


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (64, 5000, 50)         │       500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (64, 4998, 16)         │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (64, 1666, 16)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (64, 26656)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, 32)               │       853,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (64, 2)                │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,066,520 (15.51 MB)

 Trainable params: 1,355,506 (5.17 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,711,014 (10.34 MB)

In [19]:
model = Sequential()

model.add(Embedding(input_dim=top_words, output_dim=50, input_length=max_sequence_length))
model.add(Conv1D(32, 5, activation="relu", kernel_initializer=he_normal(), kernel_regularizer=l2()))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(64, activation="relu", kernel_initializer=he_normal()))
model.add(Dense(2, activation="softmax", kernel_initializer=glorot_normal()))  # 2 units for binary classification

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train_one_hot, epochs=2, batch_size=64, validation_split=0.2)
model.summary()

Epoch 1/2
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 1674s 418ms/step - accuracy: 0.6767 - loss: 0.6944 - val_accuracy: 0.7508 - val_loss: 0.5372
Epoch 2/2
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 1657s 407ms/step - accuracy: 0.7705 - loss: 0.5076 - val_accuracy: 0.7615 - val_loss: 0.5164


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (64, 5000, 50)         │       500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (64, 4996, 32)         │         8,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (64, 1665, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (64, 53280)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (64, 64)               │     3,409,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (64, 2)                │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,754,440 (44.84 MB)

 Trainable params: 3,918,146 (14.95 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,836,294 (29.89 MB)

In [21]:
from keras.layers import Embedding, Conv2D, MaxPooling2D, Flatten, Dense, Reshape
model = Sequential()

model.add(Embedding(input_dim=top_words, output_dim=50, input_length=max_sequence_length))
model.add(Reshape((max_sequence_length, 50, 1)))

model.add(Conv2D(32, kernel_size=(3, 50), activation='relu', kernel_initializer=he_normal(), kernel_regularizer=l2()))
model.add(MaxPooling2D(pool_size=(max_sequence_length - 2, 1)))
model.add(Flatten())
model.add(Dense(64, activation="relu", kernel_initializer=he_normal()))
model.add(Dense(2, activation="softmax", kernel_initializer=glorot_normal()))  # 2 units for binary classification

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train_one_hot, epochs=2, batch_size=64, validation_split=0.2)
model.summary()

Epoch 1/2
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 2364s 591ms/step - accuracy: 0.7457 - loss: 0.5754 - val_accuracy: 0.7834 - val_loss: 0.4678
Epoch 2/2
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 2321s 580ms/step - accuracy: 0.8062 - loss: 0.4334 - val_accuracy: 0.7927 - val_loss: 0.4543


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (64, 5000, 50)         │       500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (64, 5000, 50, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (64, 4998, 1, 32)      │         4,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (64, 1, 1, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (64, 32)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (64, 64)               │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (64, 2)                │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,521,224 (5.80 MB)

 Trainable params: 507,074 (1.93 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,014,150 (3.87 MB)

In [10]:
print("Shape of X_train:", X_train.shape)

Shape of X_train: (320000, 5000)


In [13]:
print(X_train[:1])

[[ 0  0  0 ... 72 22 35]]


In [14]:
print("Shape of X_train:", X.shape)

Shape of X_train: (400000,)


In [20]:
import numpy as np

num_rows, num_columns = df.shape
print(num_rows,num_columns)

400000 7
